In [33]:
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch

In [22]:
BASE_DIR = Path("/Users/gryv9001/code/nielsen/pacing-target-creation/data/raw/")
TRAIN_FILE = BASE_DIR / "train/Monthly-train.csv"

In [23]:
print(str(TRAIN_FILE))

/Users/gryv9001/code/nielsen/pacing-target-creation/data/raw/train/Monthly-train.csv


In [4]:
#data = pd.read_csv(str(DATA_FILE))

In [5]:
#data.head()

In [6]:
def read_file(file_location):
    series = []
    ids = []
    with open(file_location, 'r') as file:
        data = file.read().split("\n")

    for i in range(1, len(data) - 1):
        row = data[i].replace('"', '').split(',')
        series.append(np.array([float(j) for j in row[1:] if j != ""]))
        ids.append(row[0])

    series = np.array(series)
    return series

In [7]:
train = read_file(str(TRAIN_FILE))

In [13]:
chop_val = 72

In [14]:
def chop_series(train, chop_val):
    # CREATE MASK FOR VALUES TO BE CHOPPED
    train_len_mask = [True if len(i) >= chop_val else False for i in train]
    # FILTER AND CHOP TRAIN
    train = [train[i][-chop_val:] for i in range(len(train)) if train_len_mask[i]]
    return train, train_len_mask

In [15]:
train, train_mask = chop_series(train, chop_val)

In [18]:
len(train_mask)

48000

In [24]:
print('loading info data')
info = pd.read_csv(str(BASE_DIR / "M4info.csv"))

loading info data


In [26]:
variable = "Monthly"

In [27]:
dataInfoCatOHE = pd.get_dummies(info[info['SP'] == variable]['category'])

In [29]:
dataInfoCatOHE.tail()

,Demographic,Finance,Industry,Macro,Micro,Other
94995,0,0,0,0,0,1
94996,0,0,0,0,0,1
94997,0,0,0,0,0,1
94998,0,0,0,0,0,1
94999,0,0,0,0,0,1


In [30]:
dataInfoCatHeaders = np.array([i for i in dataInfoCatOHE.columns.values])

In [31]:
dataInfoCatHeaders

array(['Demographic', 'Finance', 'Industry', 'Macro', 'Micro', 'Other'],
      dtype='<U11')

In [36]:
dataInfoCat = torch.from_numpy(dataInfoCatOHE[train_mask].values).float()

In [37]:
dataInfoCat

tensor([[0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        ...,
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1.]])

In [ ]:
#daily.describe()

In [ ]:
sub_df = daily[["V1", "V2", "V3", "V4", "V5"]]

In [ ]:
sub_df.head(4)

In [ ]:
sub_df.dtypes

In [ ]:
sub_df = sub_df[sub_df["V2"] < 50000]
sub_df = sub_df[sub_df["V3"] < 50000]
sub_df = sub_df[sub_df["V4"] < 50000]

In [ ]:
plt.figure(figsize=(16, 5))
plt.title("V2")
x = np.arange(len(sub_df["V1"]))
plt.plot(x, sub_df["V2"], x, sub_df["V3"], x,  sub_df["V4"])
plt.grid(True);